In [ ]:
import pandas as pd
import numpy as np
import os
import librosa
import pickle
import tensorflow as tf
import librosa.display
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
import tensorflow.keras.layers as layers
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score,f1_score

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip '/content/drive/MyDrive/USG2_data.zip' -d '/content/drive/MyDrive/[라떼들]문제2번/'

unzip:  cannot find or open /content/drive/MyDrive/USG2_data.zip, /content/drive/MyDrive/USG2_data.zip.zip or /content/drive/MyDrive/USG2_data.zip.ZIP.


### 사용자 경로 지정

In [ ]:
path = "/content/drive/MyDrive/[라떼들]문제2번/" #공통된 디렉토리

In [ ]:
submit = pd.read_csv(path+'answer_sample.csv')
submit.head()

,file,label
0,test_0001.npy,-1
1,test_0002.npy,-1
2,test_0003.npy,-1
3,test_0004.npy,-1
4,test_0005.npy,-1


### origin_fre_test 파일 만들기

In [ ]:
def extract_fft_features(directory):
    global left_f
    features=[]
    sr = 16000
    for filename in os.listdir(directory):
        f = os.path.join(directory, filename)
        if os.path.isfile(f):
            data = np.load(f)
            fft = np.fft.fft(data)
            magnitude = np.abs(fft)
            freq = np.linspace(0,16000,len(magnitude))
            left_spectrum = magnitude[:int(len(magnitude)/2)]
            left_f = freq[:int(len(magnitude)/2)]
            features.append(left_spectrum)
    return features

In [ ]:
features_test=extract_fft_features(path+"test_data/")

In [ ]:
test_df = pd.DataFrame(features_test, columns=left_f)
test_df

,0.000000,0.200003,0.400005,0.600008,0.800010,1.000013,1.200015,1.400018,1.600020,1.800023,...,7998.099976,7998.299979,7998.499981,7998.699984,7998.899986,7999.099989,7999.299991,7999.499994,7999.699996,7999.899999
0,9823.0,4320.733317,5090.480612,6308.936590,6572.065325,3552.859017,8952.600822,3504.014748,1248.463304,4376.955894,...,14166.472071,14729.819729,17512.687601,13292.939526,12715.345527,11144.807039,13318.932428,6056.402328,4633.432387,11697.249334
1,13474.0,14210.477771,13853.862758,14264.096694,16243.375306,18814.005314,17780.208258,12970.950299,16075.267371,13600.819639,...,1918.791906,7696.882849,4802.228208,10186.090213,4662.230434,9007.552351,5855.244284,4384.524719,4622.289888,5308.847396
2,7401.0,1627.825446,1866.518002,4196.466417,2798.549905,5258.215999,20968.137940,29393.159932,7794.385806,31312.563026,...,2595.385883,12403.489722,9490.835402,9252.914617,18081.185649,2622.103392,10024.864701,8049.682026,2852.237632,7312.148394
3,3701.0,4459.793278,4090.076038,4126.929539,5273.128816,5759.912477,6818.099605,5685.508621,6779.759264,7042.731378,...,2386.229451,4201.311632,5489.311666,1665.734214,8000.409212,10001.086290,11841.506306,9072.166576,8156.970081,9487.482199
4,8993.0,5103.300642,4898.728207,5633.400289,5813.075860,771.374789,5089.182888,6207.556133,20566.215244,3492.097253,...,9858.782050,8568.713688,1240.427149,13283.765711,805.012718,10875.788610,6544.715683,8083.618992,4698.075213,1771.081639
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1110,21234.0,20730.249619,19120.128316,22488.467530,21704.688474,23427.363997,21386.132753,21615.701090,25415.624703,20081.251069,...,8198.205687,7540.809861,3770.604825,7621.086839,7172.335999,374.164569,9762.576717,6973.816020,4735.442010,9038.511454
1111,2696.0,4211.441451,4328.804703,3355.204984,6216.765685,6510.798879,1355.230227,5385.451104,6614.439399,3177.506384,...,24591.888232,13776.119232,9473.726662,7130.786808,13434.944545,16776.821330,2838.440128,17445.544275,8574.827493,3474.629304
1112,2079.0,1351.849978,1651.342977,1646.995014,1420.150188,1995.478058,1612.002562,630.899324,1990.750361,662.822248,...,6927.514059,15910.331824,9560.952954,2111.743705,7228.985275,5022.697746,14778.827667,10580.714694,5382.335916,4302.860959
1113,6611.0,7573.113975,10152.412777,3478.481449,15954.820841,7853.651958,19624.448386,16355.496290,32006.690523,17673.167572,...,2438.139535,5790.166978,8958.796538,5517.586075,2782.234191,3808.141603,4976.628908,6215.717552,3681.449317,4743.138701


### test_mfcc 파일 생성

In [ ]:
CFG = {
    'SR':16000,
    'N_MFCC':32, # MFCC 벡터를 추출할 개수
    'SEED':41
}

In [ ]:
from tqdm import tqdm
def get_mfcc_feature(home_directory,directory):
    features =[]
    df = pd.read_csv(home_directory+'train_label.csv')  ### 여기 경로를 지정해줘야함. (train_label 있는 곳으로..)
    for filename in tqdm(os.listdir(directory)):
        f = os.path.join(directory, filename)
        if os.path.isfile(f):
            data = np.load(f)#,sr=CFG['SR'])
            mfcc = librosa.feature.mfcc(y=data, sr=16000, n_mfcc=CFG['N_MFCC'])
            y_feature = []
            for e in (mfcc):
                y_feature.append(np.mean(e))
            features.append(y_feature)

    # # 기존의 자가진단 정보를 담은 데이터프레임에 추출된 오디오 Feature를 추가
    mfcc_df = pd.DataFrame(features, columns=['mfcc_'+str(x) for x in range(1,CFG['N_MFCC']+1)])
    df = pd.concat([df, mfcc_df], axis=1)

    print('Done.')
    return df

In [ ]:
test_mfcc = get_mfcc_feature(path,path+"test_data")

100%|██████████| 1115/1115 [00:27<00:00, 39.82it/s]


Done.


### 데이터 처리

In [ ]:
test_df = test_df.loc[:,:3000]

In [ ]:
feature = ['mfcc_2', 'mfcc_14', 'mfcc_17', 'mfcc_30', 'mfcc_12', 'mfcc_7',
       'mfcc_29', 'mfcc_22', 'mfcc_27', 'mfcc_9']
test_mfcc = test_mfcc[feature]
test_mfcc.dropna(inplace=True)

<ipython-input-14-a4f487bfd491>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_mfcc.dropna(inplace=True)


In [ ]:
test_tot = pd.concat([test_df,test_mfcc],axis=1)

In [ ]:
with open(path+'data/scaler.pkl','rb') as f:
  scaler = pickle.load(f)

In [ ]:
t_x = test_tot.values # 독립변인들의 value값만 추출
t_x = scaler.transform(t_x)
features = [test_tot.columns]
standard_scale_test = pd.DataFrame(t_x, columns=features)
standard_scale_test.head(5)

,0.0,0.2000025000312504,0.4000050000625008,0.6000075000937511,0.8000100001250016,1.0000125001562519,1.2000150001875023,1.4000175002187527,1.6000200002500031,1.8000225002812535,...,mfcc_2,mfcc_14,mfcc_17,mfcc_30,mfcc_12,mfcc_7,mfcc_29,mfcc_22,mfcc_27,mfcc_9
0,0.109082,0.046788,0.056498,0.061842,0.083907,0.040618,0.094821,0.043223,0.011266,0.023011,...,0.509692,0.364122,0.539484,0.423341,0.405149,0.352946,0.616731,0.529452,0.570763,0.542025
1,0.149634,0.154357,0.154449,0.140033,0.209442,0.219732,0.189679,0.161125,0.151856,0.072800,...,0.654380,0.386818,0.481283,0.590116,0.443079,0.565072,0.497577,0.629135,0.597768,0.463990
2,0.082181,0.017498,0.020462,0.041079,0.034926,0.060633,0.223935,0.365648,0.073335,0.168405,...,0.497133,0.388532,0.382469,0.620129,0.539285,0.647556,0.748014,0.655640,0.473963,0.526989
3,0.041085,0.048300,0.045316,0.040395,0.067046,0.066521,0.071885,0.070392,0.063715,0.037400,...,0.517762,0.502920,0.422756,0.421668,0.365492,0.574217,0.593518,0.525672,0.710809,0.626914
4,0.099863,0.055300,0.054354,0.055202,0.074055,0.007973,0.053307,0.076893,0.194440,0.018235,...,0.349527,0.453831,0.683016,0.564368,0.518107,0.178452,0.542691,0.614396,0.631322,0.378027


### 모델 로드

In [ ]:
import joblib

Model = joblib.load(path+'model/best/261(stacking,loc).pkl')
Model.predict(standard_scale_test)

/usr/local/lib/python3.9/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.2.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 1.2.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator MLPClassifier from version 1.2.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own 

array([0, 0, 0, ..., 1, 0, 0])

In [ ]:
list(Model.predict(standard_scale_test)).count(1)

261

In [ ]:
submit['label']  = Model.predict(standard_scale_test)

In [ ]:
submit['label'].value_counts()

0    854
1    261
Name: label, dtype: int64

In [ ]:
submit.to_csv('submit.csv',index=False)